In [71]:
# imports

import pandas as pd
import numpy as np
from nltk.corpus import stopwords as nltk_stopwords
from nltk.tokenize import word_tokenize as nltk_tokenize
from collections import Counter as collections_counter
import string

Spam/Ham SMS data was obtained from the Kaggle project <br>
https://www.kaggle.com/sid321axn/sms-spam-classifier-naive-bayes-ml-algo.<br>
The data was loaded into a dataframe and the messages were extracted for cleaning.

In [57]:
# load data 

_data = pd.read_csv('./spam.csv')

In [58]:
# show head

_data.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [59]:
# show shape

print(f'{_data.shape[0]} rows of messages.')

5572 rows of messages.


In [60]:
# separate text, format frame

_messages = pd.DataFrame({'messages': _data.iloc[:,1]})

In [61]:
# show messages

_messages.head()

,messages
0,"Go until jurong point, crazy.. Available only ..."
1,Ok lar... Joking wif u oni...
2,Free entry in 2 a wkly comp to win FA Cup fina...
3,U dun say so early hor... U c already then say...
4,"Nah I don't think he goes to usf, he lives aro..."


In [62]:
# convert to lower case

for _m in _messages.iterrows():
    _m[1][0] = _m[1][0].lower()

In [63]:
# check capitalization

_messages.head()

,messages
0,"go until jurong point, crazy.. available only ..."
1,ok lar... joking wif u oni...
2,free entry in 2 a wkly comp to win fa cup fina...
3,u dun say so early hor... u c already then say...
4,"nah i don't think he goes to usf, he lives aro..."


In [64]:
# remove punctuation

for _m in _messages.iterrows():
    _m[1][0] = _m[1][0].translate(str.maketrans("","", string.punctuation))

In [65]:
# check punctuation removal

_messages.head()

,messages
0,go until jurong point crazy available only in ...
1,ok lar joking wif u oni
2,free entry in 2 a wkly comp to win fa cup fina...
3,u dun say so early hor u c already then say
4,nah i dont think he goes to usf he lives aroun...


In [66]:
# tokenize

for _m in _messages.iterrows():
    _m[1][0] = nltk_tokenize(_m[1][0])

In [67]:
# check tokenization

_messages.head()

,messages
0,"[go, until, jurong, point, crazy, available, o..."
1,"[ok, lar, joking, wif, u, oni]"
2,"[free, entry, in, 2, a, wkly, comp, to, win, f..."
3,"[u, dun, say, so, early, hor, u, c, already, t..."
4,"[nah, i, dont, think, he, goes, to, usf, he, l..."


In [69]:
# remove stopwords

_stopwords = set(nltk_stopwords.words('english'))
for _m in _messages.iterrows():
    _m[1][0] = [w for w in _m[1][0] if not w in _stopwords]

In [70]:
# check stop word removal

_messages.head()

,messages
0,"[go, jurong, point, crazy, available, bugis, n..."
1,"[ok, lar, joking, wif, u, oni]"
2,"[free, entry, 2, wkly, comp, win, fa, cup, fin..."
3,"[u, dun, say, early, hor, u, c, already, say]"
4,"[nah, dont, think, goes, usf, lives, around, t..."


We needed to create a term/documnet matrix in order to conduct Baysian analysis. 

In [ ]:
# 